# 基于业务逻辑的数据清理

## 1.导入包

In [1]:
import pandas as pd 
import numpy as np
import time
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

# 2.读取数据

In [2]:
df = pd.read_csv('../data/preprocessed_1.csv',compression='gzip')

In [3]:
df['ts'] = pd.to_datetime(df['ts'])
df.sort_values(by = ['number','ts'],inplace=True)
df.reset_index(inplace=True)

In [4]:
df.head()

,index,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,mins,hour,day,month,year,dayofweek
0,2374378,2020-10-10 07:34:16,-1.0,12.975773,77.57107,12.878468,77.44533,34,7,10,10,2020,5
1,2405894,2020-10-11 08:23:42,-1.0,12.930813,77.60953,12.960320,77.58721,23,8,11,10,2020,6
2,2405895,2020-10-11 08:23:50,-1.0,12.930813,77.60953,12.960320,77.58721,23,8,11,10,2020,6
3,2405896,2020-10-11 08:23:51,-1.0,12.930813,77.60953,12.960320,77.58721,23,8,11,10,2020,6
4,2405897,2020-10-11 08:23:54,-1.0,12.930813,77.60953,12.960320,77.58721,23,8,11,10,2020,6


In [5]:
df['booking_stamp'] = df.ts.values.astype(np.int64)//10**9
df.head(50)

,index,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,mins,hour,day,month,year,dayofweek,booking_stamp
0,2374378,2020-10-10 07:34:16,-1.0,12.975773,77.571070,12.878468,77.445330,34,7,10,10,2020,5,1602315256
1,2405894,2020-10-11 08:23:42,-1.0,12.930813,77.609530,12.960320,77.587210,23,8,11,10,2020,6,1602404622
2,2405895,2020-10-11 08:23:50,-1.0,12.930813,77.609530,12.960320,77.587210,23,8,11,10,2020,6,1602404630
3,2405896,2020-10-11 08:23:51,-1.0,12.930813,77.609530,12.960320,77.587210,23,8,11,10,2020,6,1602404631
4,2405897,2020-10-11 08:23:54,-1.0,12.930813,77.609530,12.960320,77.587210,23,8,11,10,2020,6,1602404634
5,2405898,2020-10-11 08:23:56,-1.0,12.930813,77.609530,12.960320,77.587210,23,8,11,10,2020,6,1602404636
6,2406076,2020-10-11 11:57:17,-1.0,12.960213,77.587460,12.930824,77.609610,57,11,11,10,2020,6,1602417437
7,2406077,2020-10-11 11:57:31,-1.0,12.960213,77.587460,12.930824,77.609610,57,11,11,10,2020,6,1602417451
8,2500477,2020-10-16 17:51:07,-1.0,12.924353,77.549410,12.932216,77.581825,51,17,16,10,2020,4,1602870667
9,2500478,2020-10-16 17:51:25,-1.0,12.924353,77.549410,12.932216,77.581825,51,17,16,10,2020,4,1602870685


## 处理定义1：如果用户一个小时之内，在相同的经纬度，也就是同一地点，那么我们只保留同一地点同一用户的的一次叫车请求。

In [6]:
df['shift_booking_ts'] = df.groupby('number')['booking_stamp'].shift(1)
df['shift_booking_ts'].fillna(0,inplace=True)

In [7]:
df['shift_booking_ts'].astype('int64')

0                   0
1          1602315256
2          1602404622
3          1602404630
4          1602404631
              ...    
8315377    1613068718
8315378    1613158631
8315379    1613767405
8315380    1613842485
8315381    1614414383
Name: shift_booking_ts, Length: 8315382, dtype: int64

In [8]:
df.head()

,index,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,mins,hour,day,month,year,dayofweek,booking_stamp,shift_booking_ts
0,2374378,2020-10-10 07:34:16,-1.0,12.975773,77.57107,12.878468,77.44533,34,7,10,10,2020,5,1602315256,0.000000e+00
1,2405894,2020-10-11 08:23:42,-1.0,12.930813,77.60953,12.960320,77.58721,23,8,11,10,2020,6,1602404622,1.602315e+09
2,2405895,2020-10-11 08:23:50,-1.0,12.930813,77.60953,12.960320,77.58721,23,8,11,10,2020,6,1602404630,1.602405e+09
3,2405896,2020-10-11 08:23:51,-1.0,12.930813,77.60953,12.960320,77.58721,23,8,11,10,2020,6,1602404631,1.602405e+09
4,2405897,2020-10-11 08:23:54,-1.0,12.930813,77.60953,12.960320,77.58721,23,8,11,10,2020,6,1602404634,1.602405e+09


In [9]:
df['booking_time_diff_hr']  = round((df['booking_stamp'] - df['shift_booking_ts'])//3600)
df['booking_time_diff_min']  = round((df['booking_stamp'] - df['shift_booking_ts'])//60)

In [10]:
df['booking_time_diff_hr'].value_counts().to_dict()

{0.0: 4849856,
 1.0: 164914,
 23.0: 152962,
 24.0: 132454,
 2.0: 118199,
 9.0: 104854,
 3.0: 90789,
 10.0: 86794,
 8.0: 84454,
 14.0: 83715,
 13.0: 80005,
 4.0: 71379,
 11.0: 69332,
 12.0: 69267,
 7.0: 64610,
 15.0: 61892,
 5.0: 58059,
 22.0: 56709,
 6.0: 55287,
 16.0: 45075,
 21.0: 42289,
 47.0: 41525,
 25.0: 38067,
 48.0: 36781,
 20.0: 36559,
 17.0: 36124,
 19.0: 33701,
 18.0: 32710,
 71.0: 27538,
 72.0: 24658,
 26.0: 23932,
 46.0: 20902,
 27.0: 17593,
 45.0: 17142,
 49.0: 16426,
 38.0: 16319,
 95.0: 15844,
 37.0: 15186,
 39.0: 15048,
 44.0: 14897,
 96.0: 14258,
 43.0: 14004,
 28.0: 13903,
 40.0: 13586,
 70.0: 13131,
 36.0: 13019,
 42.0: 12926,
 41.0: 12516,
 50.0: 11945,
 35.0: 11789,
 29.0: 11770,
 34.0: 11479,
 33.0: 11356,
 119.0: 10863,
 62.0: 10769,
 73.0: 10731,
 32.0: 10583,
 30.0: 10279,
 69.0: 10150,
 31.0: 9997,
 120.0: 9641,
 63.0: 9561,
 61.0: 9401,
 51.0: 9313,
 68.0: 8957,
 94.0: 8381,
 143.0: 8335,
 167.0: 8252,
 64.0: 8089,
 60.0: 7865,
 67.0: 7802,
 52.0: 7755,
 144

In [11]:
df = df[~((df.duplicated(subset=['number','pick_lat','pick_lng'],keep=False))&(df.booking_time_diff_hr<=1))]

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4335828 entries, 0 to 8315381
Data columns (total 17 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   index                  int64         
 1   ts                     datetime64[ns]
 2   number                 float64       
 3   pick_lat               float64       
 4   pick_lng               float64       
 5   drop_lat               float64       
 6   drop_lng               float64       
 7   mins                   int64         
 8   hour                   int64         
 9   day                    int64         
 10  month                  int64         
 11  year                   int64         
 12  dayofweek              int64         
 13  booking_stamp          int64         
 14  shift_booking_ts       float64       
 15  booking_time_diff_hr   float64       
 16  booking_time_diff_min  float64       
dtypes: datetime64[ns](1), float64(8), int64(8)
memory usage: 595.4 MB

## 定义2：如果用户在8min之内产生了多次叫车请求，那么我们只保留1次叫车请求（这里的经纬度可能相同，也可能不同）。

In [13]:
df = df[(df.booking_time_diff_min>=8)]

## 定义3：如果用户上车和下车地点的距离小于50M，或者是行程大于500KM，那么我们直接删除这条数据。

In [16]:
%%time
def geodesic_distance(pick_lat,pick_lng,drop_lat,drop_lng):
    return round(geodesic((pick_lat,pick_lng),(drop_lat,drop_lng)).km,2)
df['geodesic_distance'] = np.vectorize(geodesic_distance)(df['pick_lat'],df['pick_lng'],df['drop_lat'],df['drop_lng'])

Wall time: 9min 22s


In [17]:
print("上车与下车点之间的距离小于50M的数据条数是:{}".format(len(df[df.geodesic_distance<=0.05])))

上车与下车点之间的距离小于50M的数据条数是:14460


In [18]:
df = df[df.geodesic_distance>0.05]

## 定义4：如果用户的上下车地点在我们的研究区域之外，我们选择删除数据。

所在地区的经纬度坐标  
印度:  ['6.2325274', '35.6745457', '68.1113787', '97.395561']  
卡纳塔克邦（Karnataka）: : ['11.5945587', '18.4767308', '74.0543908', '78.588083']  
班加罗尔: ['12.8340125', '13.1436649', '77.4601025', '77.7840515']  

In [19]:
geolocator = Nominatim(user_agent="TAXI")

In [20]:
location = geolocator.geocode("India")
location.raw

{'place_id': 307832711,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 304716,
 'boundingbox': ['6.5531169', '35.6745457', '67.9544415', '97.395561'],
 'lat': '22.3511148',
 'lon': '78.6677428',
 'display_name': 'India',
 'class': 'boundary',
 'type': 'administrative',
 'importance': 0.957689135880987,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png'}

In [21]:
boundingbox = ['6.5531169', '35.6745457', '67.9544415', '97.395561']
min_lat = float(boundingbox[0])
max_lat = float(boundingbox[1])
min_lng = float(boundingbox[2])
max_lng = float(boundingbox[3])
#这段代码的顺序是最小纬度，最大纬度，最小经度，最大经度

In [22]:
df[(df.pick_lat<=min_lat)|(df.pick_lat>=max_lat)|(df.pick_lng<=min_lng)|(df.pick_lng>=max_lng)|(df.drop_lat<=min_lat)|(df.drop_lat>=max_lat)|(df.drop_lng<=min_lng)|(df.drop_lng>=max_lng)]

,index,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,mins,hour,day,month,year,dayofweek,booking_stamp,shift_booking_ts,booking_time_diff_hr,booking_time_diff_min,geodesic_distance
9000,742031,2020-06-22 07:19:06,80.0,12.926255,77.616400,0.000000e+00,0.000000e+00,19,7,22,6,2020,0,1592810346,1.590934e+09,521.0,31271.0,8674.59
28700,2502259,2020-10-16 21:52:28,297.0,12.958342,77.517876,3.890229e+00,-3.344596e+01,52,21,16,10,2020,4,1602885148,1.602885e+09,0.0,8.0,12181.82
31484,4344697,2021-01-07 13:17:11,345.0,12.824208,77.684840,5.347187e+01,-1.021332e+02,17,13,7,1,2021,3,1610025431,1.609843e+09,50.0,3039.0,12658.51
44030,8297478,2021-03-26 20:16:01,485.0,12.932975,77.536230,2.819774e+01,1.129942e+02,16,20,26,3,2021,4,1616789761,1.616698e+09,25.0,1530.0,4043.96
56899,1786595,2020-09-09 09:36:43,641.0,12.896367,77.623800,0.000000e+00,0.000000e+00,36,9,9,9,2020,2,1599644203,0.000000e+00,444345.0,26660736.0,8675.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8216880,1985134,2020-09-21 14:40:51,98929.0,12.852040,77.675490,1.400000e-45,1.400000e-45,40,14,21,9,2020,0,1600699251,1.600356e+09,95.0,5724.0,8680.60
8230065,834555,2020-07-02 08:40:16,99078.0,12.924925,77.606125,3.772864e+01,-8.920558e+01,40,8,2,7,2020,3,1593679216,1.591804e+09,520.0,31249.0,14231.86
8232720,2331458,2020-10-06 18:12:35,99104.0,12.928082,77.603350,1.376042e+01,1.004980e+02,12,18,6,10,2020,1,1602007955,1.601834e+09,48.0,2895.0,2481.05
8253517,2747830,2020-11-02 18:57:02,99379.0,12.984081,77.593330,0.000000e+00,0.000000e+00,57,18,2,11,2020,0,1604343422,1.604310e+09,9.0,561.0,8672.39


In [23]:
df.reset_index(inplace=True,drop=True)
outside_India = df[(df.pick_lat<=min_lat)|(df.pick_lat>=max_lat)|(df.pick_lng<=min_lng)|(df.pick_lng>=max_lng)|(df.drop_lat<=min_lat)|(df.drop_lat>=max_lat)|(df.drop_lng<=min_lng)|(df.drop_lng>=max_lng)]
df = df[~df.index.isin(outside_India.index)].reset_index(drop=True)

In [24]:
df

,index,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,mins,hour,day,month,year,dayofweek,booking_stamp,shift_booking_ts,booking_time_diff_hr,booking_time_diff_min,geodesic_distance
0,2374378,2020-10-10 07:34:16,-1.0,12.975773,77.571070,12.878468,77.445330,34,7,10,10,2020,5,1602315256,0.000000e+00,445087.0,26705254.0,17.38
1,2405894,2020-10-11 08:23:42,-1.0,12.930813,77.609530,12.960320,77.587210,23,8,11,10,2020,6,1602404622,1.602315e+09,24.0,1489.0,4.06
2,2406076,2020-10-11 11:57:17,-1.0,12.960213,77.587460,12.930824,77.609610,57,11,11,10,2020,6,1602417437,1.602405e+09,3.0,213.0,4.04
3,2500477,2020-10-16 17:51:07,-1.0,12.924353,77.549410,12.932216,77.581825,51,17,16,10,2020,4,1602870667,1.602417e+09,125.0,7553.0,3.62
4,2694503,2020-10-30 09:00:44,-1.0,12.945731,77.622500,12.973030,77.616840,0,9,30,10,2020,4,1604048444,1.602871e+09,327.0,19629.0,3.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3708830,5768115,2021-02-12 19:37:11,99999.0,13.029848,77.593400,13.063751,77.589850,37,19,12,2,2021,4,1613158631,1.613069e+09,24.0,1498.0,3.77
3708831,6102760,2021-02-19 20:43:25,99999.0,13.029296,77.592580,12.927923,77.627106,43,20,19,2,2021,4,1613767405,1.613159e+09,169.0,10146.0,11.82
3708832,6137206,2021-02-20 17:34:45,99999.0,12.907576,77.600685,12.925874,77.607620,34,17,20,2,2021,5,1613842485,1.613767e+09,20.0,1251.0,2.16
3708833,6555089,2021-02-27 08:26:23,99999.0,12.956665,77.521870,12.948099,77.562990,26,8,27,2,2021,5,1614414383,1.613842e+09,158.0,9531.0,4.56


In [25]:
# 班加罗尔
location = geolocator.geocode("bangalore")
location.raw

{'place_id': 308746144,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 7902476,
 'boundingbox': ['12.8340125', '13.1436649', '77.4601025', '77.7840515'],
 'lat': '12.9767936',
 'lon': '77.590082',
 'display_name': 'Bengaluru, Bangalore North, Bangalore Urban, Karnataka, India',
 'class': 'boundary',
 'type': 'administrative',
 'importance': 0.7094348238975636,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png'}

In [26]:
boundingbox = ['12.8340125', '13.1436649', '77.4601025', '77.7840515']
min_lat = float(boundingbox[0])
max_lat = float(boundingbox[1])
min_lng = float(boundingbox[2])
max_lng = float(boundingbox[3])
#这段代码的顺序是最小纬度，最大纬度，最小经度，最大经度

In [27]:
## 有多少需求在班加罗尔上车或者下车，同时有多少乘客呢？
pck_outside_bng  = df[(df.pick_lat<=min_lat)|(df.pick_lat>=max_lat)|(df.pick_lng<=min_lng)|(df.pick_lng>=max_lng)]
drp_outside_bng  = df[(df.drop_lat<=min_lat)|(df.drop_lat>=max_lat)|(df.drop_lng<=min_lng)|(df.drop_lng>=max_lng)]

print("班加罗尔外的上车需求有{}条".format(len(pck_outside_bng)))
print("有多少乘客在班加罗尔外上车呢？{}".format(len(pck_outside_bng.number.unique())))


print("班加罗尔外的下车需求有{}条".format(len(drp_outside_bng)))
print("有多少乘客在班加罗尔外下车呢？{}".format(len(drp_outside_bng.number.unique())))


班加罗尔外的上车需求有155908条
有多少乘客在班加罗尔外上车呢？20473
班加罗尔外的下车需求有167338条
有多少乘客在班加罗尔外下车呢？26878


In [28]:
## 筛选出卡纳塔克邦之内的请求
location = geolocator.geocode("Karnataka")
location.raw


{'place_id': 307969544,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 2019939,
 'boundingbox': ['11.5945587', '18.4766494', '74.0543908', '78.588083'],
 'lat': '14.5203896',
 'lon': '75.7223521',
 'display_name': 'Karnataka, India',
 'class': 'boundary',
 'type': 'administrative',
 'importance': 0.7868060707014523,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png'}

In [29]:
boundingbox = ['11.5945587', '18.4766494', '74.0543908', '78.588083']
min_lat = float(boundingbox[0])
max_lat = float(boundingbox[1])
min_lng = float(boundingbox[2])
max_lng = float(boundingbox[3])
#这段代码的顺序是最小纬度，最大纬度，最小经度，最大经度

In [30]:
## 有多少需求在班加罗尔上车或者下车，同时有多少乘客呢？
pck_outside_KA  = df[(df.pick_lat<=min_lat)|(df.pick_lat>=max_lat)|(df.pick_lng<=min_lng)|(df.pick_lng>=max_lng)]
drp_outside_KA  = df[(df.drop_lat<=min_lat)|(df.drop_lat>=max_lat)|(df.drop_lng<=min_lng)|(df.drop_lng>=max_lng)]

print("班加罗尔外的上车需求有{}条".format(len(pck_outside_KA)))
print("有多少乘客在班加罗尔外上车呢？{}".format(len(pck_outside_KA.number.unique())))


print("班加罗尔外的下车需求有{}条".format(len(drp_outside_KA)))
print("有多少乘客在班加罗尔外下车呢？{}".format(len(drp_outside_KA.number.unique())))

班加罗尔外的上车需求有38807条
有多少乘客在班加罗尔外上车呢？6302
班加罗尔外的下车需求有39585条
有多少乘客在班加罗尔外下车呢？6917


In [31]:
# 总共有多少数据在卡纳塔克邦之外上车或者下车？

total_ride_outside_KA = df[(df.pick_lat<=min_lat)|(df.pick_lat>=max_lat)|(df.pick_lng<=min_lng)|(df.pick_lng>=max_lng)|(df.drop_lat<=min_lat)|(df.drop_lat>=max_lat)|(df.drop_lng<=min_lng)|(df.drop_lng>=max_lng)]
print("卡纳塔克邦之外的数据总量是: {}".format(len(total_ride_outside_KA)))

卡纳塔克邦之外的数据总量是: 39632


In [33]:
suspected_bad_rides = total_ride_outside_KA[total_ride_outside_KA.geodesic_distance>500]
suspected_bad_rides

,index,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,mins,hour,day,month,year,dayofweek,booking_stamp,shift_booking_ts,booking_time_diff_hr,booking_time_diff_min,geodesic_distance
105,4765341,2021-01-21 23:18:35,0.0,12.958837,77.644485,21.149794,82.782000,18,23,21,1,2021,3,1611271115,1.611220e+09,14.0,852.0,1058.39
7368,1756248,2020-09-08 14:49:08,154.0,12.922410,77.568270,25.549080,84.655800,49,14,8,9,2020,1,1599576548,1.599497e+09,22.0,1327.0,1583.00
38148,586293,2020-06-04 18:13:54,878.0,12.970724,77.582420,21.504763,80.115160,13,18,4,6,2020,3,1591294434,1.591267e+09,7.0,459.0,982.06
40544,122197,2020-04-07 11:09:24,913.0,13.004375,77.555210,19.075983,72.877655,9,11,7,4,2020,1,1586257764,1.586103e+09,43.0,2585.0,837.67
40560,288777,2020-04-29 23:11:41,913.0,13.013597,77.545740,22.837053,69.724560,11,23,29,4,2020,2,1588201901,1.588160e+09,11.0,698.0,1366.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3681746,770076,2020-06-25 11:32:12,99395.0,13.037086,77.515854,28.979435,77.689580,32,11,25,6,2020,3,1593084732,1.593016e+09,18.0,1137.0,1765.27
3681747,1086664,2020-07-29 15:29:45,99395.0,12.975827,77.605644,28.979435,77.689580,29,15,29,7,2020,2,1596036585,1.593085e+09,819.0,49196.0,1771.97
3685647,4377242,2021-01-09 14:10:14,99499.0,12.959066,77.656815,28.479555,77.079950,10,14,9,1,2021,5,1610201414,1.610187e+09,3.0,238.0,1719.45
3685648,4377307,2021-01-09 14:57:02,99499.0,12.959036,77.656770,28.479555,77.079950,57,14,9,1,2021,5,1610204222,1.610201e+09,0.0,46.0,1719.45


In [34]:
df = df[~df.index.isin(suspected_bad_rides.index)].reset_index(drop=True)
print("符合良好请求的数据量是: {}".format(len(df)))

符合良好请求的数据量是: 3708329


In [36]:
dataset = df[['ts', 'number', 'pick_lat','pick_lng','drop_lat','drop_lng','geodesic_distance','hour','mins','day','month','year','dayofweek','booking_stamp','booking_time_diff_hr', 'booking_time_diff_min']]


## 4.导出我们清好的数据

In [37]:
dataset.to_csv("../data/clean_data.csv",index=False,compression='gzip')

: 